# Mikrokosmos: a lambda calculus tutorial

## 1. Basic lambda calculus

The lambda calculus was introduced in the 1930s by Alonzo Church as a formal system for expressing the notion of computability. It is Turing-complete, that is, like Turing machines, it has the property that every computable function can be written on lambda-calculus.

It syntax is very simple; a expression can be:

  - A lambda abstraction $\lambda x. -$, which can be interpreted as a function 
    taking $x$ as an argument and returning what is written under $-$, which can
    depend on $x$.
  - A variable, which could have appeared firstly on the lambda abstraction.
  - An application of two expressions $(\lambda x. M)N$ which applies the function
    $(\lambda x. M)$ over the argument $N$. All ocurrences of $x$ on $M$ will be
    substituted by $N$. This is what is called $\beta$-reduction.

This first example defines a very simple lambda term, the identity function $\lambda x. x$, which is a function that takes an argument $x$ and returns it unchanged. And we are going to apply the function to itself. Keep in mind during this tutorial that it is perfectly possible to apply functions to functions; in fact, this is one of the core ideas of lambda calculus.

In [1]:
id = \x.x

In [2]:
id id

λa.a ⇒ id


Function application is left distributive, that is, `f g h` must be read as `(f(g))(h)` instead of `f(g(h))`. This makes it easier to write multiple argument functions such as the **constant** function, which takes two arguments and returns the first one.

In [3]:
const = \x.\y.x

In [4]:
const id id
const id const

λa.a ⇒ id
λa.a ⇒ id


A function with two arguments can be also interpreted as a function taking only one argument and returning again a function taking the other one. For example, the `(const id)` function is a function with only one argument that discards it and always returns `id`.

In [5]:
# Comments can be inserted starting a line with the # character
alwaysid = const id

In [6]:
alwaysid id
alwaysid const
alwaysid alwaysid

λa.a ⇒ id
λa.a ⇒ id
λa.a ⇒ id


A more useful example of function taking functions as arguments is the function **composition**, which takes two functions and returns a new one created by applying the two sequentially. This corresponds to the usual mathematical function composition $f \circ g$.

In [7]:
compose = \f.\g.\x.f (g x)

In [8]:
# The identity composed with the identity is again the identity.
compose id id

λa.a ⇒ id


**Exercise:** Think what should be the result of the following expressions and then check it with the interpreter.

  - `compose const id`
  - `compose id const`
  - `compose const const`

In [9]:
# -- Your solution goes here

## 2. Logic

### 2.1. The booleans

Boolean logic can be encoded in lambda calculus. Our intuition on what means to be a truth value is that it can distinghish between two values (**true** or **false**) or two branches on a program (if ... else ...).

We are going to use this intuition to write an encoding of boolean values based on their ability to choose between two branches. Maybe surprisingly, this encoding will be also useful to write the usual boolean logic gates.

In [10]:
# Church encoding of boolean truth values
true  = \a.\b.a
false = \a.\b.b

Here, a truth value is a function on two elements that chooses one of them.

 - $\mathtt{true}\ a\ b  = a$
 - $\mathtt{false}\ a\ b = b$

This is called the *Church encoding* of the booleans, as it was firstly used by Alonzo Church. This idea of defining a type based not on its content but on how it can be used will appear later, when we define more complex data structures. 

In [11]:
# Examples
true id const
false id const
true true false
false true false

λa.a ⇒ id
λa.λb.a ⇒ true, const
λa.λb.a ⇒ true, const
λa.λb.b ⇒ false, alwaysid


In particular, `true` is exactly the same lambda term as `const`.

### 2.2. If-else 

The advantage of this way of encoding the boolean values is that they can be easily used in combination with other lambda terms. In particular, the way to encode an if-else is almost trivial: it is already encoded on the lambda terms!

In [12]:
# If true, then the id function will be returned
# if false, then the const function will be returned 
(\b. b id const) true
(\b. b id const) false

λa.a ⇒ id
λa.λb.a ⇒ true, const


If we really want to write an `if-else` function, it will be, quite literally, a trivial one

In [13]:
ifelse = \b.b
(ifelse true) id const
(ifelse false) id const

λa.a ⇒ ifelse, id
λa.λb.a ⇒ true, const


### 2.3. Logic gates

Usual operations on booleans can be defined too on this encoding and they will be surprisingly easy if we think of booleans as functions choosing from two terms.

In [14]:
# The and gate takes two booleans and returns a true if and only if 
# the two given booleans are true. 
and = \p.\q.p q p

In [15]:
# Checking the truth table for the and gate
and true true
and true false
and false true
and false false

λa.λb.a ⇒ true, const
λa.λb.b ⇒ false, alwaysid
λa.λb.b ⇒ false, alwaysid
λa.λb.b ⇒ false, alwaysid


**Exercise:** Think why this definition of the `and` gate works.

*Hint: think what happens when the first argument is a `true`. What happens if it is a `false`?*

The `or` gate can be defined in a similar way.

In [16]:
# The or gate takes two booleans and returns a true if and only if
# any of them (or both) are true.
or = \p.\q.p p q

In [17]:
# Checking the truth table for the and gate
or true true
or true false
or false true
or false false

λa.λb.a ⇒ true, const
λa.λb.a ⇒ true, const
λa.λb.a ⇒ true, const
λa.λb.b ⇒ false, alwaysid


And finally, the negation operator is only a way of interchanging the two truth values

In [18]:
not = \b.b false true

In [19]:
not true
not false
not (and true true)

λa.λb.b ⇒ false, alwaysid
λa.λb.a ⇒ true, const
λa.λb.b ⇒ false, alwaysid


The boolean logic implication operator works also as a boolean gate, it can be defined as

$$(a \to b) \equiv (\neg a) \vee b,$$

that is, the implication is true if both are true or if the premise is false.

In [20]:
implies = \a.\b.or (not a) b

**Exercise:** Compute the logic table for the implication using the previous definition.

In [21]:
# -- Your solution goes here

**Exercise:** Compute the following logic clauses using lambda calculus
 
 - True or false implies false.
 - False implies that: false implies false.
 - The negation of false and the negation of true both imply true.

In [22]:
# -- Your solution goes here

**Exercise:** Define the `xor` gate as a lambda term. The `xor` of two boolean values must return a true if and only if *exactly one* of them are true. Check also its logic table.

*Hint: you may want to use the already defined `not`.*

In [23]:
# -- Your solution goes here

## 3. Church numerals and arithmetic

### 3.1. Peano and the natural numbers

In the 19th century, Giuseppe Peano gave a definition of the natural numbers and an axiomatic theory of them based on only two contructors

 - The zero is a natural number, written as Z.
 - The successor of a natural number is a natural number, written as S.
 
In those terms, the usual natural numbers will be 

$$ Z,\ SZ,\ S(SZ),\ S(S(SZ)),\ \dots $$
 
The question is now how can we encode them on lambda calculus. We do not have the ability to write the two constructors on lambda calculus, so we will make the natural numbers depend on them. This is again the same idea we used when we tried to encode booleans, we do not care about the content, but about how can we use them later.

In [24]:
# Definition of the natural numbers
0 = \s.\z.z
succ = \n.\s.\z.s (n s z)

This definition of `0` is trivial: given a successor function and a zero, return the zero. The successor function seems more complex, but it uses the same underlying idea: given a number, a successor and a zero, apply the successor to the interpretation of that number using the same successor and zero.

In [25]:
# Names of the first twenty natural numbers
1  = succ 0
2  = succ 1
3  = succ 2
4  = succ 3
5  = succ 4
6  = succ 5
7  = succ 6
8  = succ 7
9  = succ 8
10 = succ 9
11 = succ 10
12 = succ 11
13 = succ 12
14 = succ 13
15 = succ 14
16 = succ 15
17 = succ 16
18 = succ 17
19 = succ 18
20 = succ 19

Under this interpretation, a number `n` is really a function taking a function `a` as an argument and applying it `n` times over the argument `b`.

In [26]:
5

λa.λb.(a (a (a (a (a b))))) ⇒ 5


In [27]:
5 not true
4 not false

λa.λb.b ⇒ 0, false, alwaysid
λa.λb.b ⇒ 0, false, alwaysid


**Exercise:** Define a function that takes a natural number and returns true if and only if the number is even.

*Hint: you may want to interpret the given number as a function.*

In [28]:
# -- Your solution goes here

### 3.2. Addition and multiplication

The encoding of the addition and multiplication of natural numbers will profit from the interpretation of numbers as functions. This is, in fact, the only way we can use naturals; but we will quickly see that this is an strenght instead of a weakness of our encoding. We are really encoding naturals as their induction principle: we can define a function by defining a zero and a successor.

The `double` function will only change the successor for the composition of the successor function with itself.

In [29]:
double = \n.\s.\z.n (compose s s) z

In [30]:
double 0
double 3
double 4

λa.λb.b ⇒ 0, false, alwaysid
λa.λb.(a (a (a (a (a (a b)))))) ⇒ 6
λa.λb.(a (a (a (a (a (a (a (a b)))))))) ⇒ 8


**Exercise:** Define a `triple` function.

In [31]:
# -- Your solution goes here

We are going now to define **addition** using this same principle. It takes a successor and a zero, computes the first number as `(n s z)` and then uses it as a zero on the interpretation of the second one.

In [32]:
plus = \m.\n.\s.\z.m s (n s z)

In [33]:
plus 2 1
plus 3 4
plus 0 5

λa.λb.(a (a (a b))) ⇒ 3
λa.λb.(a (a (a (a (a (a (a b))))))) ⇒ 7
λa.λb.(a (a (a (a (a b))))) ⇒ 5


**Exercise:** How would you define multiplication? Keep in mind that you can use a number as a function. Keep also in mind the previous exercises on `double` and `triple`.

*Spoilers below!*

In [34]:
# -- Your solution goes here
# mymult =

There many possible ways of defining multiplication. Some of them can use the repeated application of `plus` to a number; but we are going to define **multiplication** in a way that is similar to how we defined `double` previously. We are going to interpret the successor as the n-fold application of successor.

In [35]:
mult = \m.\n.\s.\z.m (n s) z

In [36]:
mult 0 3
mult 1 5
mult 3 4

λa.λb.b ⇒ 0, false, alwaysid
λa.λb.(a (a (a (a (a b))))) ⇒ 5
λa.λb.(a (a (a (a (a (a (a (a (a (a (a (a b)))))))))))) ⇒ 12


### 3.3. The predecessor function

But, how to compute the predecessor of a number? We have not encoded negative numbers, so it could be a function returning zero whenever it tries to get the predecessor of zero. It is an insightful exercise to try to define it by yourself, but please, do not get too obsessed with it. The solution is certainly not easy. 

In [37]:
# -- You can try here
# -- Spoilers below!

The solution follows this paragraph, but you probably expected something easier! Kleene, who was a student of Alonzo Church, discovered for the first time how to write a predecessor on lambda calculus while at the dentist. This discovery made Church start thinking that every intuitively computable function could be computed using lambda calculus, that is, that the notions of lambda-computable function and intuitively computable function would coincide.

In [38]:
pred = \n.\f.\x.n (\g.(\h.h (g f))) (\u.x) (\u.u)

In [39]:
pred 4
pred 1
pred 0

λa.λb.(a (a (a b))) ⇒ 3
λa.λb.b ⇒ 0, false, alwaysid
λa.λb.b ⇒ 0, false, alwaysid


But why does something like this even work? We will develop an intuition on this kind of constructions later.

**Exercise:** Use the predecessor function to define the `minus` function. It should return the difference between two numbers. It should return zero whenever the first number is smaller than the second.

In [40]:
# -- Your solution goes here

### 3.3. Predicates on natural numbers

This encoding even allow us to write predicates on natural numbers. The first predicate will be a function distinguishing a successor from a zero. It will be user later to build more complex ones.

It is built by appliying a `const false` function `n` times to a true constant. Only if it is applied `0` times, it will return a true value.

In [41]:
iszero = \n.(n (const false) true)

In [42]:
iszero 0
iszero 2
iszero 1

λa.λb.a ⇒ true, const
λa.λb.b ⇒ 0, false, alwaysid
λa.λb.b ⇒ 0, false, alwaysid


Using this predicate, we can build `eq` and `leq`, corresponding to $==$ and $\leq$.

In [43]:
leq = \m.\n.(iszero (minus m n))
eq  = \m.\n.(and (leq m n) (leq n m))

## 4. Combinatory logic

Combinatory logic provides a notation for lambda terms independent from quantified variables. Every lambda expression can be written in terms of three combinators, $S,K,I$, which are defined as

 - $I = \lambda x.x$, the identity function.
 - $K = \lambda x.\lambda y.x$, the constant function.
 - $S = \lambda x.\lambda y.\lambda z. x z (y z)$, a generalized application.
 
The first one, the identity, can be also written as a function of $S$ and $K$.

In [44]:
I = \x.x
K = \x.\y.x
S = \x.\y.\z.(x z (y z))

In [45]:
S K K

λa.a ⇒ I, ifelse, id


The interesting property of those combinators is that every other lambda expression can be written in terms of them. We can see how a particular lambda expression is written in SKI calculus by turning on the **ski** mode of the interpreter

In [46]:
:ski on

ski mode: on


In [47]:
S
false
true
or
and

λa.λb.λc.((a c) (b c)) ⇒ S ⇒ S
λa.λb.b ⇒ KI ⇒ 0, false, alwaysid
λa.λb.a ⇒ K ⇒ K, true, const
λa.λb.((a a) b) ⇒ SII ⇒ or
λa.λb.((a b) a) ⇒ SSK ⇒ and


**Exercise:** How are Church-encoded numerals represented with SKI combinators? Compute the first four or five numbers and try to come up with the general rule.

In [48]:
# -- Your solution goes here

In [49]:
:ski off

ski mode: off


## 5. Data structures

### 5.1. Pairs

Pairs are easily defined from the boolean logic. The main idea will be that, to apply a pair to a function will be the same thing that to apply the function to its two components.

$$ \mathtt{pair}(a,b)(f) \equiv f\ a\ b $$

With this idea, pairs and their two projections are defined as follows.

In [50]:
pair = \x.\y.\z.z x y
fst = \p.p true
snd = \p.p false

We use `true` and `false` to select the first or the second argument to the function; it is possible to use the same idea to apply other functions.

In [51]:
(pair 3 4) plus
(pair true false) or

λa.λb.(a (a (a (a (a (a (a b))))))) ⇒ 7
λa.λb.a ⇒ K, true, const


### 5.2. Lists I: nil and cons

Data structures such as lists or binary trees can be represented using the same principle we used to build naturals and booleans. We would need two constructors to represent a list a `nil` signaling the end of the list and a `cons`, joining an element to the head of the list. A list would be something like this

$$ \mathtt{cons}\ 1\ (\mathtt{cons}\ 2\ (\mathtt{cons}\ 3\ \mathtt{nil})).$$

As we did with natural numbers, we are going to write a representation independent from the constructors, they are going to be passed as arguments. We need

  - `nil`, a list.
  - `cons`, a function taking an element (head) and a list (tail) and returning a new list.

In [52]:
# The interpretation of nil is the nil constructor
# The interpretation of (cons h t) is cons of h and the interpretation of t 
nil  = \c.\n.n
cons = \h.\t.\c.\n.(c h (t c n))

In [53]:
cons 1 (cons 2 (cons 3 nil))

λa.λb.((a λc.λd.(c d)) ((((λc.λd.λe.λf.((e c) ((d e) f)) λc.λd.(c (c d))) ((λc.λd.λe.λf.((e c) ((d e) f)) λc.λd.(c (c (c d)))) λc.λd.d)) a) b))


This interpretation makes easier to write folding functions for lists. We can define a function on a list simply giving the interpretation for the nil and a binary function as an interpretation for the const. For example, we can add all the elements of a list like this

In [54]:
(cons 1 (cons 2 (cons 3 nil))) plus 0

λa.λb.(a (a (a (a (a (a b)))))) ⇒ 6


It is useful to encode this principle into a function called `fold`. We are going to define a summation $\Sigma$ function and a list product $\Pi$ function on lists.

In [55]:
fold = \c.\n.\l.(l c n)

In [56]:
sum  = fold plus 0
prod = fold mult 1

In [57]:
sum  (cons 1 (cons 3 (cons 4 nil)))
prod (cons 1 (cons 3 (cons 4 nil)))

λa.λb.(a (a (a (a (a (a (a (a b)))))))) ⇒ 8
λa.λb.(a (a (a (a (a (a (a (a (a (a (a (a b)))))))))))) ⇒ 12


**Exercise:** Write the `any` and `all` functions. They are functions that can be applied over lists of booleans.

  - `all` returns true if the list is made up only of `true`s.
  - `any` returns true if there is at least one `true` on the list.

You may want to use the `fold` function.

In [58]:
# -- Your solution goes here
# all = 
# any =

**Exercise**: Write a length function using fold. The function should return the number of elements of the lists, returning 0 if the list is empty.

In [59]:
# -- Your solution goes here
# length =

### 5.3. Lists II: map and filter

Map, filter and fold are the most famous examples of higher order functions on lists and a common example of the power of functional programming, which has its roots on lambda calculus.

  - The **map** function applies a function `f` to every element on a list.
  - The **filter** function removes the elements of the list that do not satisfy a given predicate. It "filters"      
    the list, leaving only elements that satisfy the predicate.

We are going to implement these functions using our previously defined `fold`.

In [60]:
# Given a cons h t, we return a cons (f h) t; given a nil, we return a nil
map = \f.(fold (\h.\t.cons (f h) t) nil)

In [61]:
sum               (cons 1 (cons 2 (cons 3 nil)))
sum (map succ     (cons 1 (cons 2 (cons 3 nil))))
sum (map (mult 0) (cons 1 (cons 2 (cons 3 nil))))

λa.λb.(a (a (a (a (a (a b)))))) ⇒ 6
λa.λb.(a (a (a (a (a (a (a (a (a b))))))))) ⇒ 9
λa.λb.b ⇒ nil, 0, false, alwaysid


**Exercise:** Write functions

  - doubling the value of each number on a list.
  - negating each value of a list of booleans.
  

In [62]:
# -- Your solution goes here
# doublelist = 
# negate =

Filter can be defined using a boolean to decide at each step whether to return a list with a head or return the tail ignoring the head, like this

In [63]:
filter = \p.(foldr (\h.\t.((p h) (cons h t) t)) nil)

**Exercise:** Write a function that, given any list, returns a list containing only the even numbers on the list.

In [64]:
# -- Your solution goes here
# filterodd = 

### 5.4. Binary trees

Lists have been defined using two constructors and trees will be defined using the same technique. The only difference with lists is that the `cons` constructor is going to be replaced by a `node` constructor, which takes two trees as arguments. That is, a binary tree is

  - an empty tree.
  - a node, containing a label, a left subtree, and a right subtree. 

In [65]:
node = \x.\l.\r.\f.\n.(f x (l f n) (r f n))

An example of binary tree of natural numbers is the following one

In [66]:
node 4 (node 2 nil nil) (node 3 nil nil)

λa.λb.(((a λc.λd.(c (c (c (c d))))) (((((λc.λd.λe.λf.λg.(((f c) ((d f) g)) ((e f) g)) λc.λd.(c (c d))) λc.λd.d) λc.λd.d) a) b)) (((((λc.λd.λe.λf.λg.(((f c) ((d f) g)) ((e f) g)) λc.λd.(c (c (c d)))) λc.λd.d) λc.λd.d) a) b))


Defining functions using a fold-like combinator is again very simple due to the chosen representation. We are going to need also a variant of the usual function acting on three arguments, the label, the right node and the left node.

In [67]:
triplesum = \a.\b.\c.plus (plus a b) c

In [68]:
(node 4 (node 2 nil nil) (node 3 nil nil)) triplesum 0

λa.λb.(a (a (a (a (a (a (a (a (a b))))))))) ⇒ 9


## 6. Recursion

We can use and define fixpoint operators in order to define recursive
functions. The problem they have is that they can not be evaluated
without arguments into a closed form, so we have to delay the
evaluation of the expression when we bind it. To do this, we use the
`!=` operator, which binds an expression to a variable **without** simplifying it.


In [69]:
fix != (\f.(\x.f (x x)) (\x.f (x x)))

This `fix` operator allows us to use the function we are defining on its own definition. The function will be passed as the first argument to the argument of fix, as `f = fix (\f. ...)`. It is important to notice that recursive functions, even if they work, cannot be evaluated alone without entering an infinite beta-reduction loop. We need the `!=` operator when defining recursive functions to prevent them from expanding.

Our first example is the **factorial** function.

In [70]:
fact != fix (\f.\n.iszero n 1 (mult n (f (pred n))))

In [71]:
fact 3
fact 4

λa.λb.(a (a (a (a (a (a b)))))) ⇒ 6
λa.λb.(a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a (a b))))))))))))))))))))))))


Thee complexity of computing a factorial grows exponentially, and the lambda calculus (and particularly, this encoding of natural numbers) was not thought to be efficient. `fact 6` will surely be too much for the interpreter.

As a last example, we are going to define **Fibonacci** numbers.

In [72]:
fib != fix (\f.\n.iszero n 1 (plus (f (pred n)) (f (pred (pred n)))))

In [73]:
fib 0
fib 1
fib 2
fib 3
fib 4

λa.λb.(a b) ⇒ 1
λa.λb.(a (a b)) ⇒ 2
λa.λb.(a (a (a b))) ⇒ 3
λa.λb.(a (a (a (a (a b))))) ⇒ 5
λa.λb.(a (a (a (a (a (a (a (a b)))))))) ⇒ 8


## 7. Types

Until now, we have been talking about untyped lambda calculus, but we are now going to deal with the simply-typed lambda calculus. The main differences are that

  * every term has a type;
  * only a subset of the lambda expressions can be written in simply-typed lambda calculus, the typable ones;
  * every term normalizes, that is, every computation finishes;
  * as a consequence, it is not Turing-complete.
  
The command `:types on` activates types. Types are displayed with every lambda expression, but certain lambda expressions which cannot be typed cannot be used anymore. The `fix` operator is an example. 

In [74]:
:types on

types: on


In [75]:
id
K
fix

λa.a ⇒ I, ifelse, id :: A → A
λa.λb.a ⇒ K, true, const :: A → B → A
Error: non typeable expression


A type is written as a set of type variables and arrows, where `A -> B` represents the type of a function between `A` and `B`. Currying works also with types, and a multiargument function must be written as `A -> B -> C`. The interpreter will always try to infer the **most general type**, that is, it is preferible to have `A -> B` than the particular case `A -> C -> D` where `B` happens to be `C -> D`. 

In [76]:
plus
plus 3
plus 3 2

λa.λb.λc.λd.((a c) ((b c) d)) ⇒ plus :: (A → B → C) → (A → D → B) → A → D → C
λa.λb.λc.(b (b (b ((a b) c)))) :: ((A → A) → B → A) → (A → A) → B → A
λa.λb.(a (a (a (a (a b))))) ⇒ 5 :: (A → A) → A → A


### 7.1. Propositions as types

What types are inhabited? It is easy to find an expression of the type `A -> A`, but it seems that there is no expression of type `A -> B`. We can reason that any expression of that type should be able to transform any given input type onto any desired output type, and that such an expression would not be possible.

The rules of lambda calculus are similar to the rules of the intuitionistic propositional logic; this means that a type will be inhabited if and only if the type, reading arrows as logical implications, is a tautology of propositional logic.

The axioms of intuistic propositional logic are

  * every expression implies itself, `A -> A`.
  * we can discard any assumption to arrive at a conclusion `A -> B -> A`.
  * an assumption can be used multiple times to arrive at intermediate conclusions, `(A -> B -> C) -> (A -> B) -> A -> C`.
  
Those are precisely the types of the SKI combinators. As any lambda expression can be written in terms of these combinators, every lambda expression of a type is actually a **proof** of the proposition the type represents.

In [77]:
I
K
S

λa.a ⇒ I, ifelse, id :: A → A
λa.λb.a ⇒ K, true, const :: A → B → A
λa.λb.λc.((a c) (b c)) ⇒ S :: (A → B → C) → (A → B) → A → C


We can define some logical connectives using only the implication. For example, the negation of a proposition $A$ would be a function taking $A$ and returning any given type. As we discussed earlier, this should be impossible, so the existence of a function `T -> B` where `B` is a free variable should be a proof of the type `T` not being inhabited.

For example, we can write a proof of the *modus ponens* by presenting an inhabitant of the type $A \to (A \to B) \to B$, where A and B are free type variables.

In [78]:
\a.\b.b a

λa.λb.(b a) :: A → (A → B) → B


### 7.2. Products, unions and logic

Mikrokosmos supports product, union, unit and void types. They can be used by loading the library **types** as

In [79]:
:load types

Loading /home/mario/.mikrokosmos/types.mkr...


and using the following typed constructors and recursors

| Constructor | Type  |
|:-------------:|:--------:|
| `(-,-)`     | `A → B → A × B` |
| `fst` | `(A × B) → A` |
| `snd` | `(A × B) → B` |
| `inl` | `A → A + B` |
| `inr` | `B → A + B`|
| `unit` | `⊤` |
| `abort` | `⊥ → A` |
| `absurd` | `⊥ → ⊥` |
| `caseof` | `(A + B) → (A → C) → (B → C) → C` |

The following are examples of the use of typed constructors.

In [94]:
fst (2,3)
snd (2,3)
inl true
inr false
caseof (inl 3) (mult 2) (plus 1)
caseof (inr 3) (mult 2) (plus 1)
unit

λa.λb.(a (a b)) ⇒ 2 :: (A → A) → A → A
λa.λb.(a (a (a b))) ⇒ 3 :: (A → A) → A → A
(INL λa.λb.a) :: (A → B → A) + C
(INR λa.λb.b) :: A + (B → C → C)
λa.λb.(a (a (a (a (a (a b)))))) ⇒ 6 :: (A → A) → A → A
λa.λb.(a (a (a (a b)))) ⇒ 4 :: (A → A) → A → A
UNIT ⇒ unit :: ⊤


These types complete the correspondence between intuitionistic logic and lambda calculus. A type is inhabited if and only if its proposition is provable.

| Description | Type | Proposition | Description |
|:-------------|:-----:|:-----------:|---------:|
| Product type | `A × B` | `A ∧ B` | Logical conjunction |
| Disjoint union type | `A + B` | `A ∨ B` | Logical disjunction |
| Unit type | `⊤` | `⊤` | True proposition |
| Empty type | `⊥` | `⊥` | False proposition |

The characteristic difference of classical versus intuitionistic logic is that $A \vee \neg A$ and $\neg \neg A \to A$ (the law of excluded middle, LEM) are not provable on intuitionistic logic. It is not possible to find an expression of type `(A -> C) -> ((A -> F) -> C) -> C`, which would correspond to $A \vee \neg A$.

It is possible, however, to prove $\neg \neg (A \vee \neg A)$.

In [95]:
notnotlem = \f.f (\n.\m.m (\a. f (\n.\m.n a)))
notnotlem

λa.(a λb.λc.(c λd.(a λe.λf.(e d)))) ⇒ notnotlem :: (((A → B) → ((A → C) → B) → B) → C) → C


In [96]:
:types off

types: off


## 8. Libraries

Mikrokosmos comes bundled with a set of standard libraries which can be loaded using `:load std`. They provide the majority of the functions defined on this tutorial and some more.

In [97]:
:load std

Loading /home/mario/.mikrokosmos/logic.mkr...
Loading /home/mario/.mikrokosmos/nat.mkr...
Loading /home/mario/.mikrokosmos/ski.mkr...
Loading /home/mario/.mikrokosmos/datastructures.mkr...
Loading /home/mario/.mikrokosmos/fixpoint.mkr...
Loading /home/mario/.mikrokosmos/std.mkr...


In [99]:
(S plus succ) 2

λa.λb.(a (a (a (a (a b))))) ⇒ 5
